In [513]:
%matplotlib inline
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


df = pd.read_csv('df_concatenated_station_v3.csv') #import later
def get_top_50(grp):
    return grp.sort_values("IMPRESSIONS", ascending = False).head(50)




In [7]:
top50 = df.groupby('DAY')[["IMPRESSIONS", "STATION"]].apply(get_top_50).reset_index()


In [8]:
total_traffic = top50.groupby(['DAY'])['IMPRESSIONS'].sum().reset_index()


In [9]:
top50 = top50.merge(total_traffic, on = 'DAY', suffixes = ['_station', '_total'])


In [10]:
top50['density_score'] = top50['IMPRESSIONS_station'] / top50['IMPRESSIONS_total']


In [11]:
top50['slots'] = top50['density_score'].map(lambda val: round(val * 150))


In [12]:
total_slots = top50.groupby(['DAY'])[['density_score', 'slots']].sum().reset_index()


In [18]:
top50.groupby('DAY')['slots'].sum()

DAY
10    153
11    153
12    147
13    150
14    150
15    149
16    150
Name: slots, dtype: float64

In [518]:
top50[top50.DAY==12]


,DAY,level_1,IMPRESSIONS_station,STATION,IMPRESSIONS_total,density_score,slots
100,12,429,308789,34 ST-PENN STA,5059993,0.061026,9
101,12,1612,277455,GRD CNTRL-42 ST,5059993,0.054833,8
102,12,415,215529,34 ST-HERALD SQ,5059993,0.042595,6
103,12,324,209552,23 ST,5059993,0.041413,6
104,12,100,186565,14 ST-UNION SQ,5059993,0.036871,6
105,12,2437,182168,TIMES SQ-42 ST,5059993,0.036002,5
106,12,765,181054,86 ST,5059993,0.035781,5
107,12,1563,177910,FULTON ST,5059993,0.035160,5
108,12,478,157937,42 ST-PORT AUTH,5059993,0.031213,5
109,12,597,137307,59 ST,5059993,0.027136,4


In [53]:
top50.slots.loc[134]=2
top50.slots.loc[133]=3
top50.slots.loc[135]=3
top50.slots.loc[285]=3
top50.slots.loc[35]=1
top50.slots.loc[36]=1
top50.slots.loc[37]=1
top50.slots.loc[80]=1
top50.slots.loc[81]=1
top50.slots.loc[82]=1



/Users/kylemix/anaconda/lib/python3.5/site-packages/pandas/core/indexing.py:117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [220]:
def duplicate_rows(df, countcol):
    for _, row in df.iterrows():
        for i in range(int(row[countcol])-1):
            # Append this row at the end of the DataFrame
            df = pd.concat(row)
        return df


In [223]:
tester = top50[['STATION','slots']][top50.DAY==12].copy(deep=True)
# tester.set_index('STATION', inplace = True)
duplicate_rows(tester,1)

TypeError: first argument must be an iterable of pandas objects, you passed an object of type "Series"

In [517]:
performers = pd.read_csv('performer_dataset.csv')
performers.head()
performers = performers.sort_values(by='Performance_code')



In [289]:
from collections import defaultdict
def firstRound(day):
    filt = top50[['STATION','slots']][top50.DAY==day]
#     filt.set_index(['STATION','slots'])
    d= {k:v for k,v in zip(filt.STATION, filt.slots)}
    samplelist = [np.random.choice(performers.Performer_ID, replace=False) for p in performers.Performer_ID]
    newDict = defaultdict(list)
    for k,v in d.items():
        v=int(v)
        newValue = samplelist[:v]
        newDict[k]=newValue
        del samplelist[:v]
    return newDict
    


In [468]:
def make_frame(day):
    
    f = pd.DataFrame.from_dict(firstRound(day), orient = 'index').stack()
    frame =pd.DataFrame(f)
    frame.reset_index(1,drop=2,inplace=True)
    merger = top50[['STATION','IMPRESSIONS_station']][top50.DAY==day].copy(deep=True)
    merger.set_index('STATION', inplace=True)
    frame = frame.merge(merger,left_index=True,right_index=True)
    frame.reset_index(drop=True,inplace=True)
    frame.columns = ['Performer_ID','IMPRESSIONS_station']
 
    return frame

In [516]:
def run_1Week():
    df10 = make_frame(10)
    df11 = make_frame(11)
    df12 = make_frame(12)
    df13 = make_frame(13)
    df14 = make_frame(14)
    df15 = make_frame(15)
    df16 = make_frame(16)
    allTogether = pd.concat([df11,df12,df13,df14,df15,df16],axis =0)
    allTogether = allTogether.groupby('Performer_ID')['IMPRESSIONS_station'].sum().reset_index()
#     allTogether.set_index('Performer_ID',inplace = True)
    return allTogether

def run_1Month():                            
    week1 = run_1Week()
    week2 = run_1Week()
    week3 = run_1Week()
    week4 = run_1Week()
    month = week1.add(week2,axis='index').add(week3,axis='index').add(week4,axis='index')
    return month
month = run_1Month()


